In [2]:
# hashing 
import base64 

def hashbase64(user_id):   
    return base64.b64encode(user_id.encode()).decode()[:16]

In [3]:
# loading data
import pyarrow.parquet as pq
import pyarrow.compute as pc 
import polars as pl

parq = pq.ParquetFile('2022_place_canvas_history.parquet')

In [6]:
# batches 
chunk_size = 10_000_000
processed = []

for batch in parq.iter_batches(batch_size=chunk_size):
    chunk = pl.from_arrow(batch)

    chunk = chunk.with_columns(pl.col('user_id').map_elements(hashbase64, return_dtype=pl.String))

    chunk = chunk.with_columns(
        pl.col('coordinate').str.split(',').list.get(0).cast(pl.Int32).alias('x'),
        pl.col('coordinate').str.split(',').list.get(1).cast(pl.Int32).alias('y')
    )

    chunk = chunk.drop('coordinate')
    
    processed.append(chunk) 

In [7]:
# combining 
combined_df = pl.concat(processed)
# sorting time 
combined_df = combined_df.sort('timestamp') 

In [8]:
combined_df.write_parquet('2022_place_canvas_simplified.parquet')